<a href="https://colab.research.google.com/github/novoforce/Exploring-Pytorch/blob/master/new/1002_Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import torch
import torch.nn as nn #All neural network layers definitions are present
import torch.nn.functional as F #functional api for NN layers, used to adding activations
import torch.optim as optim #Optimizers are defined here
from torch.utils.data import DataLoader #Data management for NN
import torchvision.datasets as datasets #Datasets for the NN
import torchvision.transforms as transforms #Data transform for augmentation

# Create Convolutional Neural Network

How to calculate the output channels at each layer: <br/>
$$ n_{out}= [\frac{n_{in} + 2p - k}{s}] + 1  $$

$n_{in}:$ no of input features <br/>
$n_{out}:$ no of output features <br/>
$k:$ convolutional kernel size <br/>
$p:$ convolutional padding size <br/>
$s:$ convolutional stride size <br/>

In [3]:
class CNN(nn.Module):  #subclassing/inheriting from nn.Module
  def __init__(self,in_channels= 1,num_classes= 10):  #input_size= 28x28 = 784
    super(CNN,self).__init__() #calls the init function of the parent class(nn.Module)
    self.conv1= nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.pool= nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2= nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.fc1= nn.Linear(16*7*7,num_classes)

  def forward(self,x):
    x= F.relu(self.conv1(x))
    x= self.pool(x)
    x= F.relu(self.conv2(x))
    x= self.pool(x)
    x= x.reshape(x.shape[0],-1)
    x= self.fc1(x)
    return x

# Testing the class with some random generated values
# model= CNN()
# x= torch.randn((64,1,28,28))
# print(model(x).shape)


# Set Device

In [4]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [5]:
in_channel= 1
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 1

# Load Data

In [6]:
train_dataset= datasets.MNIST(root='datasets/',train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset= datasets.MNIST(root='datasets/',train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw


Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw


Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw


Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Initialize network

In [7]:
model= CNN(in_channels=1,num_classes=num_classes).to(device)

# Loss & Optimizers

In [8]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(),lr=learning_rate) 

# Train Network

In [10]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        data= data.to(device)
        targets= targets.to(device)
        # print(data.shape) #here we have to reshape the input from (64x1x28x28) to (64x764) which is basically unrolling
        # data= data.reshape(data.shape[0],-1)

        #forward propagation
        scores= model(data)
        loss= criterion(scores,targets)

        #backward propagation
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()

# Check accuracy on training & test to see how good our model is

In [12]:
def check_accuracy(loader,model):
    num_correct= 0
    num_samples= 0
    model.eval()

    with torch.no_grad():
        if loader.dataset.train:
            print("Checking accuracy on Training data")
        else:
            print("Checking accuracy on Testing data")

        for x,y in loader:
            x= x.to(device)
            y= y.to(device)
            # print('y: ',y)
            # x= x.reshape(x.shape[0],-1)
            scores= model(x)  #batch x num_classes   Here 'model' is the trained
            # print('scores: ',scores)
            _,predictions= scores.max(1)
            # print('vals: ',_,predictions)
            # break
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} %')

    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)


Checking accuracy on Training data
Got 57978/60000 with accuracy 96.63 %
Checking accuracy on Testing data
Got 9696/10000 with accuracy 96.96 %
